In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from matplotlib import pyplot as plt
import seaborn as sns
from google.colab import files
path = r'https://raw.githubusercontent.com/abdulazizHalawah/twuiaq.-project/main/Data/Data.csv'
def main():
    dataToRead=dataFrame(path)
    dataToRead = dataToRead.rename(columns={'Unnamed: 64': 'role_position'})
    DataTobeChoosen = dataToRead.loc[dataToRead['pick'] >= 1]#only real drafted players
    dataEx2021 = DataTobeChoosen.loc[DataTobeChoosen['year'] < 2021]#chosing data before 2021
    features = ['Min_per', 'ftr', 'pts', 'ORB_per', 'DRB_per', 'blk_per', 'stl_per', 'dporpag', 'porpag', 'AST_per', 'TO_per', 'FT_per']
    train_X, val_X, train_y, val_y = splitandtrain(dataEx2021,features)
    predctionOutput(train_X,train_y,val_X,val_y,dataToRead,features)
    predctionOutputBoost(train_X, train_y, val_X, val_y, dataToRead, features)
    visuals()
def dataFrame(path):
     dataToRead = pd.read_csv(path)
     return dataToRead
def dataOf2021(dataToRead):
    DataTobeChoosen = dataToRead.loc[dataToRead['pick'] >= 1]
    return DataTobeChoosen.loc[DataTobeChoosen['year'] == 2021]
def splitandtrain(dataEx2021,features):
    y = dataEx2021.pick
    X = dataEx2021[features]
    return train_test_split(X, y, random_state=1)
def predctionOutput(train_X,train_y,val_X,val_y,dataToRead,features):
    model = DecisionTreeRegressor(max_leaf_nodes=100, splitter='best', random_state=1, max_features='auto')
    model.fit(train_X, train_y)
    valdation_predctions = model.predict(val_X)
    validation_meanError = mean_absolute_error(valdation_predctions, val_y)
    DataOf2021 = dataOf2021(dataToRead)
    xOf2021 = DataOf2021[features]
    test_PredectionsOf2021 = model.predict(xOf2021)
    test_meanError = mean_absolute_error(test_PredectionsOf2021, DataOf2021.pick)
    output = pd.DataFrame( {'Name': DataOf2021.player_name, 'P': test_PredectionsOf2021, 'AP': DataOf2021.pick})
    output.to_csv('predictions2021.csv', index=False)
def predctionOutputBoost(train_X, train_y, val_X, val_y, dataToRead, features):
    gBoost = XGBRegressor(n_estimators=1200, learning_rate=0.106, max_depth=10, reg_lambda=1.1, tree_method='approx',random_state=1)
    gBoost.fit(train_X, train_y)
    Gpredections = gBoost.predict(val_X)
    gmeanError = mean_absolute_error(Gpredections, val_y)
    ball_data_2021 = dataOf2021(dataToRead)
    xOf2021 = ball_data_2021[features]
    test_PredectionsOf2021 = gBoost.predict(xOf2021)
    test_meanErrorG = mean_absolute_error(test_PredectionsOf2021, ball_data_2021.pick)
    output = pd.DataFrame(  {'Name': ball_data_2021.player_name, 'P': test_PredectionsOf2021, 'AP': ball_data_2021.pick})
    output.to_csv('xgbpredictions2021.csv', index=False)
def arrayAfterceil(df):
    X = df.P
    a = np.array(X)
    a = np.ceil(a)
    return a
def visuals():
    df = dataFrame(path)
    df = df.rename(columns={'Unnamed: 64': 'role_position'})
    DataTobeChoosen = df.loc[df['pick'] >= 1]
    dataEx2021 = DataTobeChoosen.loc[DataTobeChoosen['year'] < 2021]
    #============================================================================================
    plt.figure(figsize = (10,6))
    chart = sns.countplot(dataEx2021['conf'], palette = 'Set1')
    chart.set_xticklabels(chart.get_xticklabels(), rotation = 45)
    plt.title("Conference")
    plt.show()
#===============================================================================================
    plt.figure(figsize = (10,6))
    sns.distplot(dataEx2021['pts'], color = 'r')
    plt.title("Distribution of Points")
    plt.show()
#===============================================================================================
    plt.figure(figsize = (8,6))
    sns.countplot(dataEx2021['yr'])
    plt.title("Year of College")
    plt.show()
#===============================================================================================
    plt.figure(figsize = (8,6))
    sns.boxplot(data = dataEx2021, x = 'yr', y = 'pts', palette = 'Set1')
    plt.title("Year of College")
    plt.show()

#================================================================================================
    plt.figure(figsize = (10,6))
    sns.boxplot(data = dataEx2021, x = 'role_position', y = 'pts')
    plt.title("Role Position ~ Points")
    plt.show()
#=================================================================================================
    role_year = dataEx2021.groupby(['role_position', 'yr']).size().reset_index(name = 'Count')
    plt.figure(figsize = (12,6))
    sns.barplot(x = 'role_position', y = 'Count', hue = 'yr', data = role_year, palette = 'Set1')
    plt.title("Role Position ~ Year of College")
    plt.legend(bbox_to_anchor = (1.08, 1), borderaxespad = 0)
    plt.show()
#================================================================================================
#     path1 = r'C:\Users\abdul\PycharmProjects\NBADraft/predictions2021.csv'
#     df = dataFrame(path1)
#     y = df.AP
#     X = arrayAfterceil(df)
#     plt.figure(figsize=(12, 6))
#     sns.barplot(x=X, y=y, palette='Set1')
#     plt.title("PICKS-ACTUAL PICKS")
#     plt.legend(bbox_to_anchor=(1.08, 1), borderaxespad=0)
#     plt.show()
# #================================================================================================
#     path2 = r'C:\Users\abdul\PycharmProjects\NBADraft/xgbpredictions2021.csv'
#     df = dataFrame(path2)
#     y = df.AP
#     X = arrayAfterceil(df)
#     plt.figure(figsize=(12, 6))
#     sns.barplot(x=X, y=y, palette='Set1')
#     plt.title("PICKS-ACTUAL PICKS")
#     plt.legend(bbox_to_anchor=(1.08, 1), borderaxespad=0)
#     plt.show()
main()